<a href="https://colab.research.google.com/github/msjun23/Deep-Learning-from-Scratch/blob/main/Chapter4/1_network_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터에서 학습

**학습**이란 훈련 데이터로부터 가중치 **매개변수의 최적값을 자동**으로 획득하는 것이다. 신경망은 데이터를 보고 학습을 할 수 있다.

문제를 해결하기 위해서는 **데이터**가 가장 중요하다. 데이터에서 답을 찾고, 패턴을 발견하고, 이야기를 만드는 것을 **기계학습(Machine Learning)**이라고 한다. 여기서 사람의 개입을 더 줄인것을 **딥러닝(Deep Learning)**이라고 한다. 학습을 통한 문제 해결 방식은 아래와 같다.

> - 문제 -> 사람이 생각한 알고리즘 -> 결과
> - 문제 -> 사람이 생각한 특징(SIFT, HOG 등) -> **기계학습(SVM, KNN 등)** -> 결과
> - 문제 -> **신경망(딥러닝)** -> 결과

신경망은 이미지에 포함된 중요한 특징까지도 기계가 스스로 학습한다.

기계학습을 진행할 때 사용되는 데이터는 **훈련 데이터**(train data)와 **시험 데이터**(test data)로 나뉜다. 훈련 데이터를 통해 학습을 진행하고, 시험 데이터를 통해 앞서 훈련한 모델의 실력을 평가한다. 이렇게 과정을 나누는 이유는 처음 보는 데이터에도 적용할 수 있는 범용성을 평가하기 위함이다. 기존의 데이터가 아닌 새로운 데이터에도 적용할 수 있는 모델을 만드는 것이 기계학습의 최종 목표이다.

반대로 한 데이터셋에만 지나치게 최적화 되어 새로운 데이터셋에서 좋은 결과를 내지 못하는 상태를 **과적합, 오버피팅(overfitting)**이라고 한다.

---

# 손실 함수
신경망은 하나의 지표를 기준으로 최적의 매개변수 값을 탐색한다. 이때 사용되는 지표가 **손실 함수**(loss function)이다. 이 함수는 임의의 함수를 사용할 수도 있지만, 일반적으로는 오차제곱합과 교차 엔트로피 오차를 사용한다.

손실 함수는 해당 신경망이 얼마나 **안 좋은지**를 판단하는 지표이다.

가장 많이 쓰이는 손실 함수로는 **오차제곱합**(sum of squares for error, SSE)이다. 수식은 다음과 같다.
$$E=\frac12\sum_k(y_k-t_k)^2$$

$y_k$는 신경망의 출력, $t_k$는 정답 레이블, $k$는 데이터의 차원 수이다.

In [1]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]    # 원-핫 인코딩

In [2]:
def sum_squares_error(y, t):
  return 0.5 * np.sum((y - t) ** 2)

In [3]:
# 오차제곱합 구현
import numpy as np

# 정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1 : '2'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

# 예2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squares_error(np.array(y), np.array(t)))

0.09750000000000003
0.5975


오차제곱합 기준으로 첫 번째 추정 결과가 오차가 더 작다. 즉, 정답에 더 가까운 것으로 판단할 수 있다.

다음으로 **교차 엔트로피 오차**(cross entropy error, CEE)의 수식은 다음과 같다.
$$E=-\sum_kt_k\mathbb{log}y_k$$

여기서 로그는 밑이 $e$인 자연로그($\mathbb{log}_e$)이다. $y_k$와 $t_k$는 각각 신경망의 출력, 정답 레이블이다. 여기서 $t_k$는 원-핫 인코딩이다. 그래서 계산을 해보면 위의 식은 실질적으로 정답으로 예측한 $y_k$ 자연로그를 계산하는 식이라고 볼 수 있다.

자연로그 그래프는 입력이 1일 때 0을 출력하고, 0으로 점점 가까워질수록 값이 작아진다. 즉, 정답 레이블에 해당하는 $y_k$값이 작으면 자연로그값이 작아지지만, 앞에 -1을 곱하므로 교차 엔트로피 오차값은 커진다. 그 반대의 경우는 자연로그값이 커지고, 오차값은 작아질 것이다.

In [4]:
# 교차 엔트로피 오차 구현
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t * np.log(y + delta))   # y가 0인 경우 대처

In [5]:
# 정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


오차 엔트로피 오차의 결과 또한 오차제곱합의 결과와 동일하게 나온다.

---

기계학습 문제는 훈련 데이터에 대한 손실 함수의 값을 구하고, 그 값을 최대한 줄여주는 매개변수를 찾아낸다. 이때 모든 훈련 데이터를 대상으로 손실 함수 값을 구해야 한다. 즉, 훈련 데이터가 100개 있으면 그로부터 계산한 100개의 손실 함수 값들의 합을 지표로 삼는 것이다.

예를 들어, 모든 훈련 데이터에 N개에 대해 교차 엔트로피 오차의 합을 구하는 식은 다음과 같다.
$$E=-\frac1N\sum_n\sum_kt_{nk}\mathbb{log}y_{nk}$$

수식이 복잡해 보이지만 이전의 교차 엔트로피 오차식과 다를건 없다. 다만 N개의 데이터로 확장 후, 마지막에 N으로 나누어 정규화한 것이다. N으로 나눔으로써 **평균 손실 함수**를 구할 수 있다.

하지만 MNIST 데이터셋을 생각해보면 훈련 데이터의 수는 60,000개였다. 이것 이상으로 큰 데이터셋에 대해서 손실 함수의 합을 구하려면 시간이 꽤 걸릴 것이다. 이런 경우에는 데이터 일부를 추려 전체의 **근사치**를 구할 수 있다. 학습에서도 훈련 데이터의 일부만을 골라 학습을 수행한다. 이 일부를 **미니배치**(mini-batch)라 하고, 이런 학습 방법을 **미니배치 학습**이라 한다.

In [6]:
# 미니배치 학습 구현
import sys, os
sys.path.append(os.pardir)
import numpy as np
from mnist import load_mnist

(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(y_train.shape)

(60000, 784)
(60000, 10)


In [7]:
# 무작위로 10장만 빼내려면
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
y_batch = y_train[batch_mask]

In [8]:
# np.random.choice 예시
np.random.choice(60000, 10)

array([21761, 36527, 19169, 59522,  5139, 11861, 56959, 20732, 10463,
       43059])

위와 같이 무작위로 선택한 이 인덱스를 사용해 미니배치를 선택하면 된다.

In [9]:
# (배치용) 교차 엔트로피 오차 구현
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t * np.log(y + 1e-7)) / batch_size

y의 차원이 1차원이면(e.g. (784, )) reshape() 함수로 데이터의 형상을 바꿔준다.(e.g. (1, 784)) 그리고 배치의 크기로 나누어 정규화를 하면 이미지 1장당의 평균 교차 엔트로피 오차를 구할 수 있다.

In [10]:
# 정답 레이블이 원-핫 인코딩이 아닌 경우(숫자 레이블로 주어졌을 경우)
# 교차 엔트로피 오차 구현
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

정답 레이블이 원-핫 인코딩일 때는 정답이 아닌 레이블은 0이므로 정답에 해당하는 신경망의 출력만으로도 교차 엔트로피 오차를 계산할 수 있었다.

np.arange(batch_size)는 0부터 batch_size - 1까지의 정수 배열을 생성한다. 예를 들어 batch_size가 5, 정답 레이블 t가 [2, 7, 0, 9, 4]라면 아래와 같다.
> - np.arange(batch_size) : [0, 1, 2, 3, 4]
> - y[np.arange(batch_size), t] : [[0, 2], [1, 7], [2, 0], [3, 9], [4, 4]] 인 넘파이 배열
>
> - 위의 배열을 대입해 교차 엔트로피 오차를 구할 수 있다.

#손실 함수 -> 미분

손실 함수를 구하는 이유는 **미분**의 역할에 달려있다. 최적의 매개변수(가중치와 편향)를 탐색할 때는 손실 함수의 값을 가능한 작게 하는 매개변수 값을 찾는다. 이때 미분(기울기)를 단서로 매개변수를 서서히 갱신한다.